# 图像卷积  

### 1. 二维互相关运算  

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

def corr2d(X, K):
    """计算二维互相关运算"""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = torch.sum(X[i : i + h, j : j + w] * K)
    return Y

X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

### 2. 卷积层  
卷积层对输入和卷积核权重进行互相关运算，并在添加标量偏置之后产生输出。所以，卷积层中的两个被训练的参数是卷积核权重和标量偏置。  
高度和宽度分别为$h$和$w$的卷积核可以被称为$h\times w$卷积或$h\times w$卷积核。我们也将带有$h\times w$卷积核的卷积层称为$h\times w$卷积层。

In [2]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

### 3. 图像中的目标的边缘检测  
如下是卷积层的一个简单应用：通过找到像素变化的位置，来检测图像中不同颜色的边缘。首先，我们构造一个6 × 8像素的黑白图像。中间四列为黑色（0），其余像素为白色（1）。

In [3]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

接下来，我们构造一个高度为1、宽度为2的卷积核K。当进行互相关运算时，如果水平相邻的两元素相同，则输出为零，否则输出为非零。

In [4]:
K = torch.tensor([[1.0, -1.0]])

现在，我们对参数X（输入）和K（卷积核）执行互相关运算。如下所示，输出Y中的1代表从白色到黑色的边缘， ‐1代表从黑色到白色的边缘，其他情况的输出为0。

In [5]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

### 4. 学习卷积核  
通过X和Y来学习一个卷积核。我们先构造一个卷积层，并将其卷积核初始化为随机张量。接下来，在每次迭代中，我们比较Y与卷积层输出的平方误差，然后计算梯度来更新卷积核。为了简单起见，我们在此使用内置的二维卷积层，并忽略偏置。

In [7]:
# 构造一个二维卷积层，它具有1个输出通道和形状为（1， 2）的卷积核
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)

# 这个二维卷积层使用四维输入和输出格式（批量大小、通道、高度、宽度），
# 其中批量大小和通道数都为1
X = X.reshape(1, 1, 6, 8)
Y = Y.reshape(1, 1, 6, 7)
lr = 3e-2

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y)**2
    conv2d.zero_grad()
    l.sum().backward()
    # 迭代卷积核
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch{i+1}, loss{l.sum():.3f}')

conv2d.weight.data # 权重也是四维张量（高度，宽度，输入通道输，输出通道数）

epoch2, loss21.977
epoch4, loss7.794
epoch6, loss2.990
epoch8, loss1.191
epoch10, loss0.482


tensor([[[[ 1.0598, -0.9170]]]])

### 5. 填充  

In [8]:
def comp_conv2d(conv2d, X):
    # 这里的（1， 1）表示批量大小和通道数都是1
    X = X.reshape((1, 1) + X.shape)
    Y = conv2d(X)
    # 省略前两个维度：批量大小和通道
    return Y.reshape(Y.shape[2:])

# 请注意，这里每边都填充了1行或1列，因此总共添加了2行或2列
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1)
X = torch.rand(size=(8, 8))
print(comp_conv2d(conv2d, X).shape)

torch.Size([8, 8])


当卷积核的高度和宽度不同时，我们可以填充不同的高度和宽度，使输出和输入具有相同的高度和宽度。

In [9]:
conv2d = nn.Conv2d(1, 1, kernel_size=(5, 3), padding=(2, 1))
print(comp_conv2d(conv2d, X).shape)

torch.Size([8, 8])


### 6. 步幅  

In [10]:
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape

torch.Size([4, 4])

In [11]:
conv2d = nn.Conv2d(1, 1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
comp_conv2d(conv2d, X).shape

torch.Size([2, 2])

### 7. 多输入和多输出通道  

多输入通道是将每个通道执行互相关操作，然后将结果相加。

In [12]:
def corr2d_multi_in(X, K):
    # 先遍历“X”和“K”的第0个维度（通道维度）
    return sum([d2l.corr2d(x, k) for x, k in zip(X, K)])

X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
            [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

多输入多输出，卷积核形状为：$c_o\times c_i\times k_h\times k_w$

In [13]:
def corr2d_multi_in_out(X, K):
    return(torch.stack([corr2d_multi_in(X, k) for k in K], 0))

# 通过将核张量K与K+1（K中每个元素加1）和K+2连接起来，构造了一个具有3个输出通道的卷积核
K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [14]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

### 8. 池化层  

In [16]:
def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i ,j] = X[i: i + p_h, j: j + p_w].max()
            elif mode == 'avg':
                Y[i ,j] = X[i: i + p_h, j: j + p_w].mean()
    return Y

X = torch.ones((6, 8))
X[:, 2:6] = 0
print(X)
K = torch.tensor([[1.0, -1.0]])
Y_conv = d2l.corr2d(X, K)
print(Y_conv)
Y_pool = pool2d(Y_conv.abs(), (2, 2))
print(Y_pool) 

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])
tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])
tensor([[1., 1., 0., 0., 1., 1.],
        [1., 1., 0., 0., 1., 1.],
        [1., 1., 0., 0., 1., 1.],
        [1., 1., 0., 0., 1., 1.],
        [1., 1., 0., 0., 1., 1.]])


步幅：默认情况下，深度学习框架中的步幅与汇聚窗口的大小相同。

In [18]:
# 样本：四个维度，样本数*通道数*高*宽
X = torch.arange(16, dtype=torch.float32).reshape(1, 1, 4, 4)

pool2d = nn.MaxPool2d(3) # 步幅与汇聚窗口大小相同
print(pool2d(X))

pool2d = nn.MaxPool2d(3, stride=2)
print(pool2d(X))

pool2d = nn.MaxPool2d((2, 3), stride=(2, 3), padding=(0, 1))
print(pool2d(X))

# 多个通道
# torch.cat()与torch.stack()的区别：torch.stack()会插入新维度，torch.cat()不会
X = torch.cat((X, X + 1), 1)
pool2d = nn.MaxPool2d(3, stride=2, padding=1)
print(pool2d(X))

tensor([[[[10.]]]])
tensor([[[[10.]]]])
tensor([[[[ 5.,  7.],
          [13., 15.]]]])
tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])
